# Stock

In [1]:
import concurrent.futures
import pandas
import requests
from tqdm.notebook import tqdm as tqdm_notebook

## Utils

In [2]:
def chunks(list, n):
    # looping till length l
    for i in range(0, len(list), n):
        yield list[i : i + n]

In [3]:
BASE_URL = "https://asx.api.markitdigital.com/asx-research/1.0/companies"
URL = BASE_URL + "/directory?itemsPerPage=3000"

In [4]:
def get_statistics(symbol: str):
    try:
        KEY_URL = "{0}/{1}/key-statistics".format(BASE_URL, symbol)
        response = requests.get(KEY_URL, timeout=10)
        response_json = response.json()
        statistics = response_json.get("data", {})
        return statistics
    except Exception as exception:
        print(symbol, exception)
        return {}

In [5]:
response = requests.get(URL)
response_json = response.json()
companies = response_json.get("data", {}).get("items", [])

companies_data_frame = pandas.DataFrame(companies)
companies_data_frame.to_csv("./companies.csv", index=False, header=True)

In [6]:
companies_data_frame = pandas.read_csv("./companies.csv")
companies = companies_data_frame.to_dict("records")

chunks_companies = list(chunks(companies, 100))

all_statistics = []

for chunk_companies in tqdm_notebook(chunks_companies):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for item in chunk_companies:
            symbol = item.get("symbol", "")
            futures.append(executor.submit(get_statistics, symbol=symbol))
        for future in concurrent.futures.as_completed(futures):
            statistics = future.result()
            all_statistics.append(statistics)
    all_statistics_data_frame = pandas.DataFrame(all_statistics)
    all_statistics_data_frame.to_csv("./statistics.csv", index=False, header=True)

  0%|          | 0/21 [00:00<?, ?it/s]

SEC Error
